In [ ]:
import numpy as np

예측값 / 실제값 / 확률 불러오기

In [ ]:
audio_truth = np.load('audio_truth.npy')
text_truth = np.load('text_truth.npy')
cm_truth = np.load('cmbert_truth.npy')

In [ ]:
audio_pred = np.load('audio_pred.npy')
text_pred = np.load('text_pred.npy')
cm_pred = np.load('cmbert_pred.npy')

In [ ]:
audio_prob = np.load('audio_output.npy')
text_prob = np.load('text_output.npy')
cm_prob = np.load('cmbert_output.npy')

크기 확인

In [ ]:
print("audio")
print("실제값", audio_truth.shape)
print("예측클래스", audio_pred.shape)
print("확률", audio_prob.shape)
print("text")
print("실제값", text_truth.shape)
print("예측클래스", text_pred.shape)
print("확률", text_prob.shape)
print("cm")
print("실제값", cm_truth.shape)
print("예측클래스", cm_pred.shape)
print("확률", cm_prob.shape)

audio
실제값 (2025,)
예측클래스 (2025,)
확률 (2025, 7)
text
실제값 (2025,)
예측클래스 (2025,)
확률 (2025, 7)
cm
실제값 (2025,)
예측클래스 (2025,)
확률 (2025, 7)


## stacking 
- lgbm 모델 사용

In [ ]:
audio_text_cm_pred = np.array([audio_pred, text_pred, cm_pred])

In [ ]:
audio_text_cm_pred.shape

(3, 2025)

In [ ]:
#행이 2025개. 열이 2개로 바꿔줘야함 !!!
audio_text_cm_pred = np.transpose(audio_text_cm_pred)

In [ ]:
audio_text_cm_pred[:10]

array([[1, 0, 0],
       [3, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [4, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [4, 0, 2],
       [4, 0, 0]])

In [ ]:
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
lgbm = LGBMClassifier()
lgbm.fit(audio_text_cm_pred, audio_truth)
lgbm_pred = lgbm.predict(audio_text_cm_pred)
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, lgbm_pred)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, lgbm_pred, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, lgbm_pred, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, lgbm_pred, average="weighted")))

정확도: 0.7269
precision_score: 0.7236
recall_score: 0.7269
f1_score: 0.7168


## soft voting

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
#softmax 적용
audio_prob = softmax(audio_prob)
text_prob = softmax(text_prob)
cm_prob = softmax(cm_prob)

In [ ]:
audio_text_cm_pred_list = audio_prob + text_prob + cm_prob

In [ ]:
labels = [0,1,2,3,4,5,6]

In [ ]:
soft_voting_label = []
for i in range(len(audio_truth)):
  label = labels[audio_text_cm_pred_list[i].argmax()]
  soft_voting_label.append(label)

In [ ]:
len(soft_voting_label) #2025

2025

In [ ]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, soft_voting_label)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, soft_voting_label, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, soft_voting_label, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, soft_voting_label, average="weighted")))

정확도: 0.6899
precision_score: 0.6935
recall_score: 0.6899
f1_score: 0.6678


### weighted soft voting

In [ ]:
audio_text_cm_pred_2 = 0.1*audio_prob + 0.67*text_prob + 0.23*cm_prob

In [ ]:
wsoft_voting_label = []
for i in range(len(audio_truth)):
  label = labels[audio_text_cm_pred_2[i].argmax()]
  wsoft_voting_label.append(label)

In [ ]:
len(wsoft_voting_label) #2025

2025

In [ ]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, wsoft_voting_label)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, wsoft_voting_label, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, wsoft_voting_label, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, wsoft_voting_label, average="weighted")))

정확도: 0.6968
precision_score: 0.6969
recall_score: 0.6968
f1_score: 0.6826


## hard voting

In [ ]:
#가장 많이 나온 결과를 채택하는 코드
def mode(list, weights):
    count = 0
    mode = 0;
    for i, x in enumerate(list): 
        if list.count(x) * weights[i] > count:
            count = list.count(x) * weights[i]
            mode = x

    return mode


In [ ]:
hard_vot = []
for i in range(len(audio_pred)):
  index =  mode([audio_pred[i], text_pred[i], cm_pred[i]], [0.2, 0.5, 0.3])
  hard_vot.append(index)

In [ ]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, hard_vot)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, hard_vot, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, hard_vot, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, hard_vot, average="weighted")))

정확도: 0.6780
precision_score: 0.6745
recall_score: 0.6780
f1_score: 0.6614
